# Stage 09 — Homework Notebook

In this notebook, we implement 2 engineered features. First, we load in our dataset

In [1]:
import pandas as pd
import numpy as np

# Example synthetic data (replace with your project dataset)
# np.random.seed(0)
# n = 100
# df = pd.DataFrame({
#     'income': np.random.normal(60000, 15000, n).astype(int),
#     'monthly_spend': np.random.normal(2000, 600, n).astype(int),
#     'credit_score': np.random.normal(680, 50, n).astype(int)
# })
# df.head()

import yfinance as yf
df = yf.download(['NVDA','MSFT','AAPL','GOOG','AMZN'], period='3mo', interval='1d') # .reset_index()[['Date','Close']]
# df.columns = ['date','adj_close']

/var/folders/dk/pzmx27fj0198nc95vcbfwgn00000gn/T/ipykernel_5702/173081280.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(['NVDA','MSFT','AAPL','GOOG','AMZN'], period='3mo', interval='1d') # .reset_index()[['Date','Close']]
[*********************100%***********************]  5 of 5 completed


## Feature 1: Rolling 5-day volatility

In [7]:
df['Close_volatility_5'] = df[('Close', 'MSFT')].rolling(5).std()
print(df['Close_volatility_5'])
print(df['Close_volatility_5'].describe().T)

Date
2025-05-23         NaN
2025-05-27         NaN
2025-05-28         NaN
2025-05-29         NaN
2025-05-30    4.274891
                ...   
2025-08-18    4.519893
2025-08-19    4.991219
2025-08-20    7.069140
2025-08-21    6.776104
2025-08-22    4.818718
Name: Close_volatility_5, Length: 63, dtype: float64
count    59.000000
mean      4.008117
std       2.025340
min       1.071679
25%       2.833903
50%       3.445455
75%       4.550804
max      10.844931
Name: Close_volatility_5, dtype: float64


Here, we took the last 5 closing prices and calculated the standard deviation. We see here that the highest rolling 5 day has a max value of 10.845 - this marks an extremely high amount of volatility in a short timespan. 

### Rationale for Feature 1
From our EDA, we noted that there was a lack of involvement of time as a factor as well as how there wasn't any noticeable outliers found in intraday (Close - open price) movements. We opted to explore the rolling 5-day volatility instead to account for both of those shortcomings. This allows us to explore short-term price instability better as any significant shift is more likely to take place over a longer stretch than over the course of a single day. By calculating this instead of simply one day shifts, we were able to capture a noticeable spike of $10.85 in standard deviation, which we can identify and investigate further. 

## Feature 2: MSFT volume trading spikes flag 

In [17]:
# Rolling mean and std
rolling_mean = df[('Volume', 'MSFT')].rolling(window=10, min_periods=10).mean()
rolling_std = df[('Volume', 'MSFT')].rolling(window=10, min_periods=10).std()

# Calculate threshold
threshold = rolling_mean + 2 * rolling_std

# Create spike flag column
df[('VolumeSpike', 'MSFT')] = (df[('Volume', 'MSFT')] > threshold).astype(int)

print(df[('VolumeSpike','MSFT')].sum())


msft_columns = [col for col in df.columns if col[1] == 'MSFT']

# Filter the rows where the spike flag is 1, and only keep MSFT columns   #ChatGPT used here to print the table
msft_spikes = df[df[('VolumeSpike', 'MSFT')] == 1][msft_columns]
print(msft_spikes)


3
Price            Close        High         Low        Open    Volume  \
Ticker            MSFT        MSFT        MSFT        MSFT      MSFT   
Date                                                                   
2025-06-20  476.616486  482.666537  476.087357  481.438576  37576200   
2025-07-30  512.397644  515.103218  508.603893  514.324469  26380400   
2025-07-31  532.624390  554.538376  531.027040  554.318706  51617300   

Price      VolumeSpike  
Ticker            MSFT  
Date                    
2025-06-20           1  
2025-07-30           1  
2025-07-31           1  


### Rationale for Feature 2
This is useful for our model because we are now able to flag when higher-than-average volumes occurred for MSFT. This is useful because volume spikes historically correlate with wider price fluctuations, so we are now able to see when these shifts occur and investigate the underlying causes further. 